---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [95]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    univ_towns = pd.read_csv('university_towns.txt',header=None,names=['Data'])
    univ_towns['Match'] = univ_towns['Data'].str.contains('edit')    
    univ_towns['State'] = univ_towns['Data'].where(univ_towns['Match'] == True)
    univ_towns['State']=univ_towns['State'].str.replace('\[edit]', '')
    states_iter = univ_towns['State'].dropna().values    
    rangeInSt = univ_towns['Match'].values
    i = 0
    final_states = [states_iter[i]]
    for j in range(1,len(rangeInSt)):
        if rangeInSt[j] == True:
            i+=1
            final_states.append(states_iter[i])
        else:
            i+=0
            final_states.append(states_iter[i])
    univ_towns['State'] = final_states
    univ_towns['RegionName'] = univ_towns['Data'].where(univ_towns['Match'] == False)
    univ_towns= univ_towns.dropna()
    univ_towns['RegionName'] = univ_towns['RegionName'].str.replace(pat=r'\s\(.*', repl='')
    univ_towns = univ_towns.drop(['Data','Match'],1)
    df_states = pd.DataFrame({
      "State": list(states.values()),
      "Acronyms": list(states.keys())
    })
    answer = pd.merge(univ_towns,df_states,how='left')
    return answer.head(50)

get_list_of_university_towns()

,State,RegionName,Acronyms
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
5,Alabama,Troy,AL
6,Alabama,Tuscaloosa,AL
7,Alabama,Tuskegee,AL
8,Alaska,Fairbanks,AK
9,Arizona,Flagstaff,AZ


In [46]:
def get_recession_start():
  
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel('gdplev.xls',skiprows=219,usecols=('e:g'))
    data.columns = ['Quarter','b','GDP']
    data = data.drop(['b'],1)
    gdp_iter = data['GDP'].values
    init = 0
    for i in range (0,len(gdp_iter)):
        if gdp_iter[i] - gdp_iter[i-1]<0:
            init = gdp_iter[i-1] - gdp_iter[i-2]
            if init < 0:
                #print(i)
                start = i-1
                break               
    start_res = data['Quarter'].loc[start]
    return start_res
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():   
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel('gdplev.xls',skiprows=219,usecols=('e:g'))
    data.columns = ['Year/Quarter','b','GDP']
    data = data.drop(['b'],1)
    gdp_iter = data['GDP'].values
    init = 0
    for i in range (33,len(gdp_iter)):
        if gdp_iter[i] - gdp_iter[i-1]>0:
            init = gdp_iter[i-1] - gdp_iter[i-2]
            if init > 0:
                #print(i)
                end = i
                break
    end_res = data['Year/Quarter'].loc[end]
    return end_res
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel('gdplev.xls',skiprows=219,usecols ='e:g')
    data.columns = ['Year/Quarter','b','GDP']
    data = data.drop(['b'],1)
    data = data[33:40]
    bottom = data.loc[data['GDP'] == data['GDP'].min(),'Year/Quarter'].iloc[0]
    return  bottom
get_recession_bottom()

'2009q2'

In [91]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data = data.drop(['RegionID','Metro','CountyName','SizeRank'],1)
    data = data.set_index(['State','RegionName'])
    data = data.loc[:,'2000-01':'2016-09']
    housing = (data.groupby(pd.PeriodIndex(data.columns, freq='Q'), axis=1)
                                                                           .mean()
                                                                           .rename(columns=lambda x: str(x).lower()))

    housing = housing.reset_index()
    df_states = pd.DataFrame({
      "State": list(states.values()),
      "Acronyms": list(states.keys())
    })
    #df_states = df_states.set_index('State')

    answer = pd.merge(df_states,housing, how = 'inner',left_on=['Acronyms'] ,right_on = ['State'])
    answer = answer.drop(['State_y','Acronyms'], 1)
    answer = answer.rename(columns = {'State_x':'State'}).set_index(['State','RegionName'])
    return answer
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State    RegionName                                                  
Ohio     Columbus       94366.666667   95833.333333   97133.333333   
         Cleveland      67466.666667   67933.333333   68300.000000   
         Toledo         65100.000000   67000.000000   68533.333333   
         Akron          70100.000000   71666.666667   71866.666667   
         Dayton                  NaN            NaN            NaN   
...                              ...            ...            ...   
Virginia Linden        121700.000000  125200.000000  125800.000000   
         Bumpass       127300.000000  127966.666667  128866.666667   
         Greenville    114833.333333  116966.666667  115200.000000   
         Mount Sidney  137766.666667  136866.666667  134500.000000   
         Henrico       128566.666667  130766.666667  132266.666667   

                              2000q4         2001q1         2001q2  \
State    RegionName                                                  
Ohio     Columbus       98266.666667   99400.000000  100266.666667   
         Cleveland      68700.000000   68933.333333   69233.333333   
         Toledo         70100.000000   70800.000000   71800.000000   
         Akron          72300.000000   72833.333333   73533.333333   
         Dayton                  NaN            NaN            NaN   
...                              ...            ...            ...   
Virginia Linden        122366.666667  122700.000000  123633.333333   
         Bumpass       130833.333333  132866.666667  136333.333333   
         Greenville    114033.333333  119733.333333  120266.666667   
         Mount Sidney  133633.333333  136100.000000  139800.000000   
         Henrico       133266.666667  135233.333333  136733.333333   

                              2001q3         2001q4         2002q1  \
State    RegionName                                                  
Ohio     Columbus      101066.666667  102200.000000  103400.000000   
         Cleveland      69566.666667   69966.666667   70566.666667   
         Toledo         73300.000000   73833.333333   74600.000000   
         Akron          73933.333333   74066.666667   74500.000000   
         Dayton                  NaN            NaN            NaN   
...                              ...            ...            ...   
Virginia Linden        126400.000000  130033.333333  134600.000000   
         Bumpass       137933.333333  140566.666667  143600.000000   
         Greenville    123000.000000  124166.666667  122566.666667   
         Mount Sidney  143200.000000  145066.666667  142000.000000   
         Henrico       138600.000000  141333.333333  143533.333333   

                              2002q2  ...         2014q2         2014q3  \
State    RegionName                   ...                                 
Ohio     Columbus      104800.000000  ...  103133.333333  104500.000000   
         Cleveland      71233.333333  ...   52166.666667   52033.333333   
         Toledo         75200.000000  ...   55066.666667   55700.000000   
         Akron          75133.333333  ...   62800.000000   62766.666667   
         Dayton                  NaN  ...   51133.333333   50333.333333   
...                              ...  ...            ...            ...   
Virginia Linden        137700.000000  ...  177500.000000  176866.666667   
         Bumpass       146100.000000  ...  214533.333333  218900.000000   
         Greenville    122833.333333  ...  161700.000000  160233.333333   
         Mount Sidney  143700.000000  ...  217966.666667  212333.333333   
         Henrico       146133.333333  ...  201633.333333  204000.000000   

                              2014q4         2015q1         2015q2  \
State    RegionName                                                  
Ohio     Columbus      106433.333333  107866.666667  109433.333333   
         Cleveland      52500.000000   52333.333333   52333.333333   
         Toledo         56000.000000   55

In [94]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing = convert_housing_data_to_quarters()
    univ_towns = get_list_of_university_towns()
    df_states = pd.DataFrame({
        "State": list(states.values()),
        "Acronym": list(states.keys())
    })
    univ_towns = pd.merge(univ_towns,df_states,how='left')#.drop('State',1).rename(columns = {'Acronym':'State'})
    start_res = get_recession_start()
    end_res = get_recession_end()
    res_bottom = get_recession_bottom()    
    housing = housing.loc[:,'2008q3':res_bottom].drop('2009q1',axis = 1)
    housing = housing.reset_index()
    housing['Price Ratio'] = housing['2008q3'] / housing[res_bottom]  
    univ_housing = pd.merge(univ_towns,housing,how = 'outer',left_on = ['State','RegionName'],right_on = ['State','RegionName'],indicator = True)
    university_towns_values = univ_housing[(univ_housing['_merge'] == 'both')]
    non_university_towns_values = univ_housing[(univ_housing['_merge'] == 'right_only')]
    university_town = university_towns_values['Price Ratio']
    non_university_town = non_university_towns_values['Price Ratio']
    test = ttest_ind(university_town,non_university_town,nan_policy='omit')

    differet = None
    p = test[1]
    better = None
    if test[1] < 0.01:
        differet = True
    else:
        differet = False
    if test[0] < 0:
        better = 'university town'
    else:
        better = 'non-university town'
    answer = (differet,p,better)
    
    return test
run_ttest() 

Ttest_indResult(statistic=-2.7582703861390776, pvalue=0.005821494237532901)

In [50]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def get_list_of_university_towns():
  """
  Returns a DataFrame of towns and the states they are in from the 
  university_towns.txt list. The format of the DataFrame should be:
  DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
  columns=["State", "RegionName"]  )
    
  The following cleaning needs to be done:
  1. For "State", removing characters from "[" to the end.
  2. For "RegionName", when applicable, removing every character from " (" to the end.
  3. Depending on how you read the data, you may need to remove newline character '\n'. 
  """
  df = pd.read_table('https://storage.googleapis.com/um_ds_intro/university_towns.txt', header=None)
  df.columns = ['StateRegion']
  df['StateFlag'] = df['StateRegion'].str.contains(pat=r'\[edit\]')
  df_states = df[df['StateFlag'] == True]
  # 找出不重複州名 -------------
  States = df_states['StateRegion'].str.replace(pat=r'\[edit\]', repl='').unique()
  StateFlags = df['StateFlag'].values
  j = 0
  StateMatchRegion = [States[j]]
  # 分隔州名與區域名的邏輯 -------------
  for i in range(1, len(StateFlags)):
    if StateFlags[i] == True:
      j += 1
      StateMatchRegion.append(States[j])
    else:
      j += 0
      StateMatchRegion.append(States[j])
  # 清理資料 -------------
  df['StateMatchRegion'] = StateMatchRegion
  df['RegionName'] = df['StateRegion'].str.replace(pat=r'\s\(.*', repl='')
  ans = df[df['StateFlag'] == False]
  ans = ans.drop(['StateRegion', 'StateFlag'], axis=1)
  ans.columns = ['State', 'RegionName']
  mapping_df = pd.DataFrame({
      "State": list(states.values()),
      "StateAcronyms": list(states.keys())
  })
  ans = pd.merge(ans, mapping_df, how='left')
  return ans
get_list_of_university_towns()

,State,RegionName,StateAcronyms
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
...,...,...,...
512,Wisconsin,River Falls,WI
513,Wisconsin,Stevens Point,WI
514,Wisconsin,Waukesha,WI
515,Wisconsin,Whitewater,WI
